# 01 - Context Managers in Python

#### Introduction

(PEP 343)

Typically, the steps involved in using a context manager are the following:

1. Create a context (a minimal amount of state needed for a block of code)
2. Execute some code that uses variables from the context
3. Automatically clean up the context when we're done

Let's take a look at a simple example:

```python
with open('test.txt', 'w') as f:
    print(f.readlines())
```

- `open()`: **Creates** the context
- `with`: **enters** the context

The context closes once the indented block completes.

Context managers data in our scope **on entry** and **on exit**. 

Therefore, it's very useful for anything that needs the **Enter/Exit** or **Start/Stop** or **Set/Reset** pattern.

Examples are:
- Open / close a file.
- Start db transaction / commit or abort transaction.
- Set decimal precision to 3 / reset back to original precision
- Change / reset (the state of a class)
- Lock / release (a thread)

#### Lecture

The functionality of a context manager can be achieved with a `try`-`finally` block, but it gets quite cumbersome.

The layout is as following:
```python
with <context object> as <obj_name>:
    <with block>
<after with block; context cleaned up>
```

A `context` object is one that implements the `context management protocol`.

Classes implement this protocol by implementing two methods:
1. `__enter__`: does the setup and optionally returns some object
2. `__exit__` tear down / cleanup

Consider this context:
```python
with CtxManager() as obj:
    # do something
# done with context
```
Let's replicate the functionality of a context manager with a simplified `try`-`finally`:
```python
mgr = CtxManager()
obj = mgr.__enter__()
try:
    # do something
finally:
    # done with context
    # mgr.__exit__()
```

**Further Breakdown**

In the following code, note that `MyClass` behaves like a regular class from which objects can be instantiated until it encounters a `with` keyword.

- **It is `with` that calls the `__enter__()` method.**
- The return of `__enter__()` is passed to the object after the `as` keyword.
- To emphasise, what gets put into `obj` is **not** the return of `MyClass` but rather the return of `__enter__()`.
- `MyClass().__exit__()` only gets called after the `with` block **or** if an exception is called - the arguments of `__exit__()` are for dealing with exceptions.

```python
class MyClass:
    def __init__(self):

    def __enter__(self):
        # return obj

    def __exit__(self, <...>):
        # clean up

my_obj = MyClass()

with MyClass() as obj:
    # do something
```

**Scopes**

A `with` block does **not** have its own scope unlike a function or list comprehension. It just shares the scope of wherever it's running in. Even the `obj` in `as obj` is within this same scope.

To illustrate with a simple example:
```python
# module.py
with open(fname) as f:
    row = next(f)

print(f, row) # this will work
```
`f` and `row` are symbols in the **global** scope even after the context closes.

Note that, in the case of `open()`, the return of `open()` is the same as the return of its `__enter__()` method. This allows both approaches:
- `f = open('test.txt', 'w')`
- `with open('test.txt', 'w') as f:`

**`__enter__` and `__exit__`**

`__enter__` is quite simple; we've explained everything that we need to about it.

`__exit__` will **always** run even if an exception occurs, but if that's the case, how do we deal with it in comparison to when we normally finish our `with` block? 

    We can silence it which lets it continue the regular teardown or we can propagate it which lets it continue as if we hadn't intercepted it. The latter will terminate the entire module program thereby preventing any code below the `with` block to execute, unlike the former which will let the code continue as normal.

    The `__exit__` method needs 3 arguments:
    - exception type
    - exception value
    - traceback object (the big output that retraces our steps when we hit an exception) 

    If no exception occurs, then all three of these arguments will return `None`.

    The `__exit__` method must return `True` or `False`. 
    - `True`: silence any raised exception; continue execution as normal
    - `False`: propagate exception

#### Coding

##### Example 1

Here's a basic example:

In [5]:
class MyContext:
    def __init__(self):
        self.obj = None

    def __enter__(self):
        print('Entering context...')
        self.obj = 'The Return Object'
        return self.obj

    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context...')
        if exc_type:
            print(f"*** Error occurred, manually catching: {exc_type}, {exc_value}")
        return False

Note:
- `exc_type`, `exc_value`, `exc_tb` can be named whatever we like, but the positions matter. The first for the type, the second for the value and the third for the traceback info object.
- Think of `__exit__` as *silent* or *normal* exit. Then, `return True` will silence the exception and no traceback will appear. `return False` will trap any exceptions caught and bubble it back up.

In [8]:
# since `return False`, we expect exception to bubble back up and remaining code after `with` block to not run

with MyContext() as obj:
    print(f"Inside block; {obj = }")
    raise ValueError('some value')

print('rest of program here; I wont run')

Entering context...
Inside block; obj = 'The Return Object'
exiting context...
*** Error occurred, manually catching: <class 'ValueError'>, some value


ValueError: some value

Now let's switch the return flag to `False`:

In [10]:
class MyContext:
    def __init__(self):
        self.obj = None

    def __enter__(self):
        print('Entering context...')
        self.obj = 'The Return Object'
        return self.obj

    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context...')
        if exc_type:
            print(f"*** Error occurred, manually catching: {exc_type}, {exc_value}")
        return True

In [11]:
# since `return True`, we expect exception to be suppressed and remaining code after `with` block will run

with MyContext() as obj:
    print(f"Inside block; {obj = }")
    raise ValueError('some value')

print('rest of program here; I will run')

Entering context...
Inside block; obj = 'The Return Object'
exiting context...
*** Error occurred, manually catching: <class 'ValueError'>, some value
rest of program here; I will run


**Good Practices**

Often we will trap certain common exceptions, handle them and silent exit `return True`, but for the uncommon exceptions, we'll let them bubble back up (`return False`).

##### Example 2

Let's create a context manager that creates and utilises a new resource upon entering the context:

In [21]:
class Resource:
    def __init__(self, name):
        self.name = name
        self.state = None

class ResourceManager:
    def __init__(self, name):
        self.name = name
        self.resource = None  # we only want to create the resource when we enter a context

    def __enter__(self):
        print('entering context')
        self.resource = Resource(self.name)
        self.resource.state = 'Created'
        return self.resource

    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context')
        self.resource.state = 'Destroyed'
        if exc_type: # if an exception has occured
            print('Exception occurred!')
        return False

In [25]:
with ResourceManager('spam') as res:
    print(f"{res.name = }, {res.state = }")

print(f"{res.name = }, {res.state = }")

entering context
res.name = 'spam', res.state = 'Created'
exiting context
res.name = 'spam', res.state = 'Destroyed'


# 02 - Caveat with Lazy Iterators

As we know, `open()` creates a context and utilises a lazy iterator. What might often happen is the context closes before we get a chance to iterate through the iterator. For example:

In [30]:
import csv

def read_data():
    with open("nyc_parking_tickets_extract.csv", 'w') as f:
        return csv.reader(f, delimiter=',', quotechar='"')

reader = read_data()
for row in reader:
    print(row)

ValueError: I/O operation on closed file.

This occurred because `return` is present in the `with` block. As a result, `with` called `open`'s `__exit__()` which closed the file. How do we get around this?

After `return` returns, there's nothing left to execute in the `with` block so the `context` can exit.

But `yield`/`yield from` doesn't exit until we've iterated through the entire iterator and hit a `StopIteration` exception. This is how we can solve our problem.

Note: `csv.reader()` is an iterator; we can either start a `for` loop that yields on each iteration or yield directly from the iterator with `yield from`.

The two lines below are *basically* identical

```python
yield from csv.reader(f, delimiter=',', quotechar='"')`
```
```python
for row in csv.reader(f, delimiter=',', quotechar='"'):
    yield row
```

In [33]:
import csv

def read_data():
    with open("nyc_parking_tickets_extract.csv") as f:
        yield from csv.reader(f, delimiter=',', quotechar='"')

reader = read_data()

for row in reader:
    print(row)
    break

['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description']


# 03 - Not Just a Context Manager

There's not much in this section. Just know that you can implement other protocols in addition to the **context manager protocol**. For example, we can implement the **iterator** and the **context manager protocol**:

In [34]:
class DataIterator:
    def __init__(self, fname):
        self._fname = fname
        self._f = None
    
    def __iter__(self):
        return self
    
    def __next__(self):
        row = next(self._f)
        return row.strip('\n').split(',')
    
    def __enter__(self):
        self._f = open(self._fname)
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        if not self._f.closed:
            self._f.close()
        return False

In [35]:
with DataIterator('nyc_parking_tickets_extract.csv') as data:
    for row in data:
        print(row)
        break

['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description']


But this isn't kitted to support the iterator independently (though it wouldn't be difficult to do):

In [36]:
data = DataIterator('nyc_parking_tickets_extract.csv')
for row in data:
    print(row)
    break

TypeError: 'NoneType' object is not an iterator

# 04 - Additional Uses

Some more additional use cases besides those mentioned earlier are:
- starting and stopping a timer to calculate how long the `with` block takes to run
- acquire thread lock, perform some operations and release thread lock
- redirect `stdout` to a file, perform some operations that write to `stdout` and reset `stdout` back to original value
- format text/code with tags on either end e.g. prints `<p>`, waits for some text and then prints `</p>` -> `<p>some text</p>` These context managers can be nested.
- A list maker with its own `.print` implemented. Every time the context manager is entered, we prepend an indent character. We can next `with` blocks to produce multi-indented print statements.

#### Example 1 - Decimal precision

Decimals have a context which can be used to define many things, such as precision, rounding mechanism, etc.

By default, Decimals have a "global" context - i.e. one that will apply to any Decimal object by default:

In [1]:
import decimal

In [2]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [4]:
import decimal

class precision:
    def __init__(self, prec):
        self.prec = prec
        self.original_prec = decimal.getcontext().prec

    def __enter__(self):
        decimal.getcontext().prec = self.prec

    def __exit__(self, exc_type, exc_value, exc_tb):
        decimal.getcontext().prec = self.original_prec
        return False

with precision(3):
    print(decimal.Decimal(1)/decimal.Decimal(3))

print(decimal.Decimal(1)/decimal.Decimal(3))

0.333
0.3333333333333333333333333333


In fact, the decimal class already has a context manager, and it's way better than ours, because we can set not only the precision, but anything else we want:

In [5]:
with decimal.localcontext() as ctx:
    ctx.prec = 3
    print(decimal.Decimal(1) / decimal.Decimal(3))
print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.3333333333333333333333333333


So this is an example of using a context manager for a **Change - Reset** type of situation.

#### Example 2 - Timer

This is an example of a **start** - **stop** situation:

In [10]:
from time import perf_counter, sleep

class Timer:
    def __init__(self):
        self.elapsed = 0

    def __enter__(self):
        self.start = perf_counter()
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

with Timer() as t:
    sleep(1)

t.elapsed

1.0046027000062168

#### Example 3 - Redirecting stdout

By default, print redirects to the jupyter kernal, but we can change this to a file very easily.

We can view what the current stdout is with:

In [3]:
import sys

print(sys.stdout)

We want to make a context manager that takes a file name and redirects all print statements to that file. We need to make sure we close the file in the exit:

In [16]:
class OutToFile:
    def __init__(self, fname):
        self.fname = fname
        self.original_std = sys.stdout
   
    def __enter__(self):
        self.file = open(self.fname, 'w') 
        sys.stdout = self.file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self.original_std
        self.file.close()
        return False

with OutToFile('test.txt'):
    print('Line 1')
    print('Line 2')

print('Line 3')

with open('test.txt', 'r') as f:
    print(f.readlines())

Line 3
['Line 1\n', 'Line 2\n']


#### Example 4 - Re-entrant Context Manager (ListMaker)

This is a more complex example where we have the ability to enter a context recursively (contexts within contexts).

This context manager will allow us to create a list that automatically indents every time we (re)enter a context. The final output that we're looking for will look something like this:
```
Title
- Item 1
   - Sub item 1a
   - Sub item 1b
      - subsub item 1b(i)
      - subsub item 1b(ii)
- Item 2
   - Sub item 2a
   - Sub item 2b
```

In [15]:
class ListMaker:
    def __init__(self, title, prefix, indent=3):
        self.title = title
        self.prefix = prefix
        self.indent = indent
        self.current_indent = 0
        print(title)

    def __enter__(self):
        self.current_indent += self.indent
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.current_indent -= self.indent

    def print_item(self, arg):
        s = ' ' * self.current_indent + self.prefix + str(arg)
        print(s)

with ListMaker('My Title', prefix='- ') as lm:
    lm.print_item('Item 1')
    with lm:
        lm.print_item('Subitem 1a')
        lm.print_item('Subitem 1b')
        with lm:
            lm.print_item('Subitem 1b(i)')
            lm.print_item('Subitem 1b(ii)')
    lm.print_item('Item 2')
    with lm:
        lm.print_item('Sub item 2a')
        lm.print_item('Sub item 2b')          

My Title
   - Item 1
      - Subitem 1a
      - Subitem 1b
         - Subitem 1b(i)
         - Subitem 1b(ii)
   - Item 2
      - Sub item 2a
      - Sub item 2b


We can redirect this output to a file using our `OutToFile` context manager:

In [24]:
with OutToFile('my_list.txt'):
    with ListMaker('My Title', prefix='- ') as lm:
        lm.print_item('Item 1')
        with lm:
            lm.print_item('Subitem 1a')
            lm.print_item('Subitem 1b')
            with lm:
                lm.print_item('Subitem 1b(i)')
                lm.print_item('Subitem 1b(ii)')
        lm.print_item('Item 2')
        with lm:
            lm.print_item('Sub item 2a')
            lm.print_item('Sub item 2b')  

In [26]:
with open('my_list.txt', 'r') as f:
    for row in f:
        print(row, end='')

My Title
   - Item 1
      - Subitem 1a
      - Subitem 1b
         - Subitem 1b(i)
         - Subitem 1b(ii)
   - Item 2
      - Sub item 2a
      - Sub item 2b


# 05 - Generators and Context Managers

One of the design goals of context managers was to enable the creation of a context manager from a generator function.

Generators and context managers have a number of similarities: A setup, the central code, and an end. 

The setup for a generator is the initial yield which pauses the generator. The end is reached by further yielding until we reach a `StopIteration` exception.

As a basic example of opening a file using a generator that mimics a context manager:

In [39]:
# generator function
def open_file(fname, mode):
    f = open(fname, mode)
    try:
        # this does nothing but gives us the file object returned by open.
        # It does NOT call __enter__ or anything; that requires `with`.
        yield f
    
    finally:
        f.close()

# call gen func `ctx` as it behaves like one
ctx = open_file('test.txt', 'r')
f = next(ctx)

try:
    # main work with file
    print(f.readlines())

finally:
    try:
        next(ctx)
        
    except StopIteration:
        # we expect the exception so we don't need to do anything
        pass

['Line 1\n', 'Line 2\n']


The generator function can be as generic as we like as long as it follows a particular pattern:
```python
def gen(args):
    # do setup
    try:
        yield <obj>
    finally:
        # clean up <obj>
```

The generic pattern to get this generator to be treated as a context manager is the following:
```python
ctx = gen(...)
obj = next(gen)

try:
    # do work with <obj>
finally:
    try:
        next(ctx)
    except StopIteration:
        pass
```
This generic pattern to get a context manager replicate from a generator can be implemented using a context manager itself!
```python
class GenContext:
    def __init__(self, gen):
        self.gen = gen()

    def __enter__(self):
        obj = next(self.gen)
        return obj

    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self.gen)
        except StopIteration:
            pass
        return False
```
This is now quite clean; we can create a context manager out of any generator:
```python
gen = open_file('test.txt', 'w')
with GenContext(gen) as f:
    f.readlines()
```

Here's a basic example:

In [45]:
class GenCtxManager:
    def __init__(self, gen, *args, **kwargs):
        self._gen = gen(*args, **kwargs)
    
    def __enter__(self):
        return next(self._gen)

    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self._gen)
        except StopIteration:
            pass

        return False

In [46]:
def my_gen():
    my_list = [1, 2, 3, 4]
    try:
        print('creating pseudo context')
        yield my_list
    finally:
        print('exiting pseudo context')

with GenCtxManager(my_gen) as obj:
    print(obj)

creating pseudo context
[1, 2, 3, 4]
exiting pseudo context


In [49]:
def open_file(fname, mode):
    f = open(fname, mode)
    try:
        print('opening file...')
        yield f
    
    finally:
        print('closing file...')
        f.close()

with GenCtxManager(open_file, 'test.txt', 'r') as f:
    print(f.readlines())

opening file...
['Line 1\n', 'Line 2\n']
closing file...


# 06 - The contextmanager Decorator

#### Lecture

We saw how to create a context manager from a generator function, where the generation function had the following form:
```python
def gen_function(args):
    # do setup
    try:
        yield <obj>  # similar to return value of `__enter__`
    finally:
        # do cleanup / __exit__
```
Take note of the 3 parts to the generator function which mimics the 3 parts of a context manager.

Then, we created our own context manager called `GenCtxManager` which takes our generator function and provides a handle to the yielded object. 

One issue with this approach is we lose the name of the thing we actually care about (the generator function). 

That is, we must write 

```
gen = gen_func(args)
with GenCtxManager(gen_function)
```
as opposed to `with gen_function` or something like it, so our clarity suffers.

So, instead, we can use a decorator to hide/encapsulate these steps. 

In [51]:
class GenCtxManager:
    def __init__(self, gen_obj):
        self._gen = gen_obj
    
    def __enter__(self):
        return next(self._gen)

    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self._gen)
        except StopIteration:
            pass

        return False

def contextmanager_dec(gen_fn):
    def inner(*args, **kwargs):
        gen = gen_fn(*args, **kwargs)
        return GenCtxManager(gen)
    return inner

This `contextmanager_dec` will be applied to a generator function and return a context manager. Therefore, the generator function post-decoration will be a context manager that can be entered with `with <generator_func_name>`.

In [53]:
@contextmanager_dec
def open_file(fname, mode):
    f = open(fname, mode)
    try:
        print('opening file...')
        yield f
    
    finally:
        print('closing file...')
        f.close()

`open_file` is now a context manager that has wrapped a generator function. We can treat it like a regular context manager:

In [54]:
with open_file('test.txt', 'r') as f:
    print(f.readlines())

opening file...
['Line 1\n', 'Line 2\n']
closing file...


Of course, this decorator is already implemented in the standard library under `contextlib.contextmanager`. It is far better though, as it has more robust exception handling.

**To summarise, `contextlib.contextmanager` is a decorator which turns a generator function of the form *`try-yield-finally`* (see above) into a context manager**.

#### Example 1 - Open File

Let's apply the standard libary decorator to our generator function to ensure it works all the same:

In [55]:
from contextlib import contextmanager

@contextmanager
def open_file(fname, mode):
    f = open(fname, mode)
    try:
        print('opening file...')
        yield f
    
    finally:
        print('closing file...')
        f.close()

with open_file('test.txt', 'r') as f:
    print(f.readlines())

opening file...
['Line 1\n', 'Line 2\n']
closing file...


#### Example 2 - Revisit Timer

Instead of writing a timer context manager and then using it directly, we are going to write a timer generator function, convert it to a context manager with the decorator, and then use that. 

We need to make sure we have the `try-yield-finally` pattern inside out generator function. Here's one way to do it:

In [57]:
from time import sleep, perf_counter

@contextmanager
def timer():
    stats = dict()
    stats['start'] = perf_counter()
    
    try:
        yield stats

    finally:
        stats['end'] = perf_counter()
        stats['elapsed'] = stats['end'] - stats['start']

with timer() as stats:
    sleep(1)

print(stats)

{'start': 496731.2352724, 'end': 496732.2432664, 'elapsed': 1.0079939999850467}


Comparing this with the original...
```python
class Timer:
    def __init__(self):
        self.elapsed = 0

    def __enter__(self):
        self.start = perf_counter()
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

with Timer() as t:
    sleep(1)

t.elapsed
```
... and the generator function may be considered simpler.

#### Example 3 - Revisit redirecting stdout

Same idea as above:

In [61]:
import sys

@contextmanager
def out_to_file(fname):
    original_stdout = sys.stdout
    file = open(fname, 'w')
    sys.stdout = file
    try:
        yield None
        
    finally:
        file.close()
        sys.stdout = original_stdout

with out_to_file('test_2.txt'):
    print('Line X')
    print('Line Y')

with open('test_2.txt', 'r') as f:
    print(f.readlines())
    
print('Line Z')

['Line X\n', 'Line Y\n']
Line Z


It turns out the standard library already has an implementation of redirecting stdout called `redirect_stdout` which is much more generic than ours.

Ours was forced to take a file name, whereas `redirect_stdout` needs an output stream. A file object is an example of an output stream. 

In [62]:
from contextlib import redirect_stdout

with open('test_3.txt', 'w') as f:
    with redirect_stdout(f):
        print('Look on the bright side of life')

with open('test_3.txt', 'r') as f:
    print(f.readlines())

['Look on the bright side of life\n']


# 07 - Nested Context Managers